# Querying Data with Session-Scoped UDTFs

This notebook demonstrates various ways to query data using session-scoped UDTFs.

## Prerequisites

- UDTFs must be registered (see `basic_registration.ipynb`)
- CDF credentials configured in Secret Manager


## Step 1: Query Single UDTF


In [ ]:
# Basic query with LIMIT
query = """
SELECT * FROM smallboat_udtf(
    client_id => SECRET('cdf_sailboat_sailboat', 'client_id'),
    client_secret => SECRET('cdf_sailboat_sailboat', 'client_secret'),
    tenant_id => SECRET('cdf_sailboat_sailboat', 'tenant_id'),
    cdf_cluster => SECRET('cdf_sailboat_sailboat', 'cdf_cluster'),
    project => SECRET('cdf_sailboat_sailboat', 'project'),
    name => NULL,
    description => NULL
) LIMIT 10;
"""

result = spark.sql(query)
result.show(truncate=False)


## Step 2: Select All Properties vs Specific Properties


In [ ]:
# Select all properties (SELECT *)
query_all = """
SELECT * FROM smallboat_udtf(
    client_id => SECRET('cdf_sailboat_sailboat', 'client_id'),
    client_secret => SECRET('cdf_sailboat_sailboat', 'client_secret'),
    tenant_id => SECRET('cdf_sailboat_sailboat', 'tenant_id'),
    cdf_cluster => SECRET('cdf_sailboat_sailboat', 'cdf_cluster'),
    project => SECRET('cdf_sailboat_sailboat', 'project'),
    name => NULL,
    description => NULL
) LIMIT 10;
"""

result_all = spark.sql(query_all)
print("All properties:")
result_all.show(truncate=False)


In [ ]:
# Select specific properties (subset of columns)
query_subset = """
SELECT 
    external_id,
    name,
    space
FROM smallboat_udtf(
    client_id => SECRET('cdf_sailboat_sailboat', 'client_id'),
    client_secret => SECRET('cdf_sailboat_sailboat', 'client_secret'),
    tenant_id => SECRET('cdf_sailboat_sailboat', 'tenant_id'),
    cdf_cluster => SECRET('cdf_sailboat_sailboat', 'cdf_cluster'),
    project => SECRET('cdf_sailboat_sailboat', 'project'),
    name => NULL,
    description => NULL
) LIMIT 10;
"""

result_subset = spark.sql(query_subset)
print("Specific properties only:")
result_subset.show(truncate=False)


## Step 3: Named vs Positional Parameters


In [ ]:
# Named parameters (recommended - clearer and more maintainable)
query_named = """
SELECT * FROM smallboat_udtf(
    client_id => SECRET('cdf_sailboat_sailboat', 'client_id'),
    client_secret => SECRET('cdf_sailboat_sailboat', 'client_secret'),
    tenant_id => SECRET('cdf_sailboat_sailboat', 'tenant_id'),
    cdf_cluster => SECRET('cdf_sailboat_sailboat', 'cdf_cluster'),
    project => SECRET('cdf_sailboat_sailboat', 'project'),
    name => 'MyBoat',
    description => NULL
) LIMIT 10;
"""

result_named = spark.sql(query_named)
print("Named parameters:")
result_named.show(truncate=False)


## Step 4: Time Series UDTF Queries


In [ ]:
# Register time series UDTFs first (if not already registered)
from pyspark.sql.functions import udtf
from cognite.pygen_spark.time_series_udtfs import (
    TimeSeriesDatapointsUDTF,
    TimeSeriesDatapointsLongUDTF,
    TimeSeriesLatestDatapointsUDTF,
)

time_series_datapoints_udtf = udtf(TimeSeriesDatapointsUDTF)
time_series_datapoints_long_udtf = udtf(TimeSeriesDatapointsLongUDTF)
time_series_latest_datapoints_udtf = udtf(TimeSeriesLatestDatapointsUDTF)

print("✓ Time Series UDTFs registered")


In [ ]:
# Query single time series datapoints
query_ts = """
SELECT * FROM time_series_datapoints_udtf(
    space => 'sailboat',
    external_id => 'vessels.urn:mrn:imo:mmsi:258219000::129038::navigation.speedOverGround',
    start => '1d-ago',
    end => 'now',
    client_id => SECRET('cdf_sailboat_sailboat', 'client_id'),
    client_secret => SECRET('cdf_sailboat_sailboat', 'client_secret'),
    tenant_id => SECRET('cdf_sailboat_sailboat', 'tenant_id'),
    cdf_cluster => SECRET('cdf_sailboat_sailboat', 'cdf_cluster'),
    project => SECRET('cdf_sailboat_sailboat', 'project')
) ORDER BY timestamp LIMIT 10;
"""

result_ts = spark.sql(query_ts)
result_ts.show(truncate=False)


In [ ]:
# Query multiple time series in long format
query_ts_long = """
SELECT * FROM time_series_datapoints_long_udtf(
    space => 'sailboat',
    external_ids => 'vessels.urn:mrn:imo:mmsi:258219000::129038::navigation.speedOverGround,vessels.urn:mrn:imo:mmsi:258219000::129038::navigation.courseOverGroundTrue',
    start => '1d-ago',
    end => 'now',
    client_id => SECRET('cdf_sailboat_sailboat', 'client_id'),
    client_secret => SECRET('cdf_sailboat_sailboat', 'client_secret'),
    tenant_id => SECRET('cdf_sailboat_sailboat', 'tenant_id'),
    cdf_cluster => SECRET('cdf_sailboat_sailboat', 'cdf_cluster'),
    project => SECRET('cdf_sailboat_sailboat', 'project')
) ORDER BY time_series_external_id, timestamp LIMIT 20;
"""

result_ts_long = spark.sql(query_ts_long)
result_ts_long.show(truncate=False)


In [ ]:
# Query latest datapoints for multiple time series
query_ts_latest = """
SELECT * FROM time_series_latest_datapoints_udtf(
    space => 'sailboat',
    external_ids => 'vessels.urn:mrn:imo:mmsi:258219000::129038::navigation.speedOverGround,vessels.urn:mrn:imo:mmsi:258219000::129038::navigation.courseOverGroundTrue',
    before => 'now',
    include_status => true,
    client_id => SECRET('cdf_sailboat_sailboat', 'client_id'),
    client_secret => SECRET('cdf_sailboat_sailboat', 'client_secret'),
    tenant_id => SECRET('cdf_sailboat_sailboat', 'tenant_id'),
    cdf_cluster => SECRET('cdf_sailboat_sailboat', 'cdf_cluster'),
    project => SECRET('cdf_sailboat_sailboat', 'project')
) ORDER BY time_series_external_id;
"""

result_ts_latest = spark.sql(query_ts_latest)
result_ts_latest.show(truncate=False)
